In [3]:
import pandas as pd

In [ ]:
df =pd.read_csv("airports.csv")
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
unique_types = df["type"].unique()

In [ ]:
filtered_airports = df[df["type"].isin(["medium_airport", "large_airport"])]
filtered_airports.head()

In [ ]:
search_result = df[df.applymap(lambda x: 'LAX' in str(x)).any(axis=1)]
print(search_result)

In [ ]:
row_2214 = df[df['id'] == 2214]

# Display the row
print(row_2214)

In [ ]:
iata = df[df['iata_code'] == "JFK"]

# Display the row
print(iata)

In [ ]:
data = {'origin': ["HAM"],
        'destination': ["LAX"]}

df = pd.DataFrame(data)

# Write the DataFrame to a CSV file
df.to_csv('output.csv', index=False)

print("CSV file 'output.csv' has been created successfully.")

In [24]:
own=pd.read_csv("data/own.csv")
print(own.head())

   Airline  Aircraft origin destination  origin_lat  origin_lng  \
0      NaN       NaN    HAM         LAX   53.630402    9.988230   
1      NaN       NaN    HAM         FRA   53.630402    9.988230   
2      NaN       NaN    LAX         FRA   33.942501 -118.407997   
3      NaN       NaN    IAH         PHX   29.984400  -95.341400   
4      NaN       NaN    CPT         HHN         NaN         NaN   

   destination_lat  destination_lng     distance  Regestration  Seat  \
0        33.942501      -118.407997  9086.586271           NaN   NaN   
1        50.036521         8.561268   411.451527           NaN   NaN   
2        50.036521         8.561268  9321.278881           NaN   NaN   
3        33.435302      -112.005905  1620.425103           NaN   NaN   
4              NaN              NaN          NaN           NaN   NaN   

   Flight Number   Date   Home link   Wikipedia link  
0            NaN    NaN         NaN              NaN  
1            NaN    NaN         NaN              NaN  

In [ ]:
import pandas as pd

# Sample data
# airport_data = {
#     'code': ['LAX', 'JFK', 'SFO'],
#     'latitude_deg': [33.9425, 40.6413, 37.6217],
#     'longitude_deg': [-118.4081, -73.7781, -122.3755]
# }

# own_data = {
#     'origin': ['LAX', 'JFK', 'SFO'],
#     'destination': ['JFK', 'SFO', 'LAX']
# }

# Create DataFrames
airport_df = pd.read_csv("airports.csv")
own_df = pd.read_csv("output.csv")

def get_coordinates(airport_code):
    # Get latitude and longitude for given airport code
    row = airport_df[airport_df['iata_code'] == airport_code].iloc[0]
    return row['latitude_deg'], row['longitude_deg']

def get_flight_coordinates(row):
    # Extract airport codes from 'origin' and 'destination' columns
    origin_code = row['origin']
    destination_code = row['destination']
    
    # Get coordinates for origin and destination airports
    origin_lat, origin_lng = get_coordinates(origin_code)
    destination_lat, destination_lng = get_coordinates(destination_code)
    
    return origin_lat, origin_lng, destination_lat, destination_lng

# Apply the function to each row in own_df
own_df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = own_df.apply(get_flight_coordinates, axis=1, result_type='expand')

print(own_df)


In [15]:
#from here
import pandas as pd

airport_df = pd.read_csv("data/airports.csv")
own_df = pd.read_csv("data/own.csv")

def get_coordinates(airport_code):
    # Get latitude and longitude for given airport code
    row = airport_df[airport_df['iata_code'] == airport_code].iloc[0]
    return row['latitude_deg'], row['longitude_deg']

def get_flight_coordinates(row):
    # Extract airport codes from 'origin' and 'destination' columns
    origin_code = row['origin']
    destination_code = row['destination']
    
    # Get coordinates for origin and destination airports
    origin_lat = row['origin_lat'] if pd.notnull(row['origin_lat']) else get_coordinates(origin_code)[0]
    origin_lng = row['origin_lng'] if pd.notnull(row['origin_lng']) else get_coordinates(origin_code)[1]
    destination_lat = row['destination_lat'] if pd.notnull(row['destination_lat']) else get_coordinates(destination_code)[0]
    destination_lng = row['destination_lng'] if pd.notnull(row['destination_lng']) else get_coordinates(destination_code)[1]
    
    return origin_lat, origin_lng, destination_lat, destination_lng

# Apply the function to each row in own_df
own_df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = own_df.apply(get_flight_coordinates, axis=1, result_type='expand')

print(own_df)


  origin destination  origin_lat  origin_lng  destination_lat  \
0    HAM         LAX   53.630402     9.98823        33.942501   

   destination_lng     distance  more_info  
0      -118.407997  9086.586271        NaN  


In [16]:
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the great-circle distance between two points on the Earth."""
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of Earth in kilometers
    return c * r

# Iterate over rows
for index, row in own_df.iterrows():
    if pd.isnull(row['distance']):
        lat1, lon1, lat2, lon2 = row['origin_lat'], row['origin_lng'], row['destination_lat'], row['destination_lng']
        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            own_df.at[index, 'distance'] = calculate_distance(lat1, lon1, lat2, lon2)

print(own_df)
own_df.to_csv('output.csv', index=False)

  origin destination  origin_lat  origin_lng  destination_lat  \
0    HAM         LAX   53.630402     9.98823        33.942501   

   destination_lng     distance  more_info  
0      -118.407997  9086.586271        NaN  


In [17]:
import pandas as pd

def add_row_to_dataframe(df, values_dict):
    # Create a new DataFrame with the values from the dictionary
    new_row = pd.DataFrame([values_dict])

    # Append the new row to the original DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

    return df

# Example usage:
# Suppose you have an existing DataFrame
#df = pd.DataFrame(columns=['A', 'B', 'C'])

# Dictionary containing values for the new row
new_row_values = {'origin': "HAM", 'destination': "JFK",}

# Add the new row to the DataFrame
own_df = add_row_to_dataframe(own_df, new_row_values)
# df.to_csv('output.csv', index=False)
print(own_df)


  origin destination  origin_lat  origin_lng  destination_lat  \
0    HAM         LAX   53.630402     9.98823        33.942501   
1    HAM         JFK         NaN         NaN              NaN   

   destination_lng     distance  more_info  
0      -118.407997  9086.586271        NaN  
1              NaN          NaN        NaN  


In [20]:
def get_column_sum(df, column_name):
    column_sum=df[column_name].sum()
    return column_sum
sum_kilo=get_column_sum(own_df,'distance')
print(sum_kilo)

9086.586271240469


In [22]:
def surrounded_world(distance):
    surrounded=distance/40030
    return surrounded

surrounden=surrounded_world(40000)
print(surrounden)


0.9992505620784412


In [23]:
df = pd.read_csv("data/own.csv")
def most_common_value(df, column_name):
    """Find the most common value in a column and its frequency."""
    # Count the occurrences of each value in the column
    value_counts = df[column_name].value_counts()
    
    # Find the value that occurs the most
    most_common_value = value_counts.idxmax()
    
    # Find the frequency of the most common value
    frequency = value_counts.max()
    
    return most_common_value, frequency
most_common, frequency = most_common_value(df, "origin")
print(f"The most common value in column 'A' is {most_common}, which occurs {frequency} times.")

The most common value in column 'A' is HAM, which occurs 2 times.


In [42]:
df=pd.read_csv("data/kolj.csv")
pd.set_option('display.max_columns', None)
df.columns = df.columns.str.strip()
df = df.iloc[:, :-7]
print(df)
df.to_csv("data/kolj2.csv",index=False)

              Airline   Aircraft Regestration   Seat Flight Number Origin  \
0         Lufthansa\t   A321-200          NaN     9E        LH 005   EDDH   
1   Lufthansa           B747-8i        D-ABYF    28K        LH 430   EDDF   
2              United       B767          NaN    38L       UA 1989   KORD   
3              United     E175LR       N85355    21B       UA 6248   KIAH   
4              United  ERJ-145XR       N12163    10A       UA 5019   KCMH   
5              United      B767           NaN    40A         UA 30   KEWR   
6   Lufthansa           A319-100          NaN     6F    LH 015\r\n   EDDH   
7              United   B777-200          NaN    43L        UA 047   EDDF   
8              United   B777-200          NaN  35L\t         UA 46   KIAH   
9   Lufthansa           A320-200          NaN  18A\t        LH 028   EDDF   
10          Eurowings   A319-100       D-AGWX     6F       EW 7180   EDDH   
11             United   B777-200          NaN    37A         UA 21    AMS   

In [43]:
df = df.drop(columns=['Unnamed: 8', 'Unnamed: 9'])
df.to_csv("data/kolj2.csv",index=False)

In [45]:
# Add new columns with NaN values
import numpy as np
df['origin_lat'] = np.nan
df['origin_lng'] = np.nan
df['destination_lat'] = np.nan
df['destination_lng'] = np.nan
df['distance'] = np.nan

In [47]:
df['origin_link'] = np.nan
df['origin_wikipedia_link'] = np.nan
df['destination_link'] = np.nan
df['destination_wikipedia_link'] = np.nan

In [48]:
df.to_csv("data/kolj2.csv")
print(df)

              Airline   Aircraft Regestration   Seat Flight Number Origin  \
0         Lufthansa\t   A321-200          NaN     9E        LH 005   EDDH   
1   Lufthansa           B747-8i        D-ABYF    28K        LH 430   EDDF   
2              United       B767          NaN    38L       UA 1989   KORD   
3              United     E175LR       N85355    21B       UA 6248   KIAH   
4              United  ERJ-145XR       N12163    10A       UA 5019   KCMH   
5              United      B767           NaN    40A         UA 30   KEWR   
6   Lufthansa           A319-100          NaN     6F    LH 015\r\n   EDDH   
7              United   B777-200          NaN    43L        UA 047   EDDF   
8              United   B777-200          NaN  35L\t         UA 46   KIAH   
9   Lufthansa           A320-200          NaN  18A\t        LH 028   EDDF   
10          Eurowings   A319-100       D-AGWX     6F       EW 7180   EDDH   
11             United   B777-200          NaN    37A         UA 21    AMS   

In [4]:
flights_df=pd.read_csv("data/kolj2.csv")
airports_df=pd.read_csv("data/airports.csv")

In [14]:
row = airports_df[airports_df['iata_code']=='LAX'].iloc[:, :]
print(row)
print(type(row))

         id ident           type                               name  \
37768  3632  KLAX  large_airport  Los Angeles International Airport   

       latitude_deg  longitude_deg  elevation_ft continent iso_country  \
37768     33.942501    -118.407997         125.0       NaN          US   

      iso_region municipality scheduled_service gps_code iata_code local_code  \
37768      US-CA  Los Angeles               yes     KLAX       LAX        LAX   

                     home_link  \
37768  https://www.flylax.com/   

                                          wikipedia_link     keywords  
37768  https://en.wikipedia.org/wiki/Los_Angeles_Inte...  Tom Bradley  
<class 'pandas.core.frame.DataFrame'>


In [27]:
for index, row in flights_df.iterrows():
    origin_value = row['origin']
    lax_row = airports_df.loc[airports_df['ident'] == origin_value]
    latitude = lax_row['latitude_deg'].iloc[0] 
    longitude = lax_row['longitude_deg'].iloc[0]
    print(latitude)
    # Now you can use origin_value as needed
    #print(origin_value)

EDDH
EDDF
KORD
KIAH
KCMH
KEWR
EDDH
EDDF
KIAH
EDDF
EDDH
AMS
KIAH
KPHX
KIAH
KAUS
EDDF
EDDH
EDDF
FACT
FACT
EDDF


In [25]:
lax_row = airports_df.loc[airports_df['ident'] == 'EDDF']
latitude = lax_row['latitude_deg'].iloc[0] 
longitude = lax_row['longitude_deg'].iloc[0]
print(longitude)
print(latitude)

8.561268
50.036521
